In this notebook, the Toronto neighborhoods are explored. 

In [1]:
import pandas as pd

In [2]:
# Read in the Toronto neighborhoods file and the file of Geospatial coords for each postal code.
# The Geocoder did not work out for me and since time was running out, I decided to use the provided file of
# geospatial coordinates.

TorPostCodedf = pd.read_csv('TorontoProject2.csv')
TorLatLongdf = pd.read_csv('Geospatial_Coordinates.csv')

In [3]:
#Look at TorPostCodedf
TorPostCodedf.head(5)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [4]:
#Look at TorLatLongdf (made from the file of geospatial coords)
TorLatLongdf.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
#Delete Postal Code column in preparation for merging dataframes.
subsetdf = TorLatLongdf[['Latitude', 'Longitude']]
subsetdf.head()

,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476


In [6]:
#Since both dataframes are sorted by Postcode, the dataframe of the neighborhoods
#can be merged with the dataframe of latitudes and longitudes using pandas concat.
TorExploredf = pd.concat([TorPostCodedf, subsetdf], axis=1)
TorExploredf.head(10)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


The third part of the assignment starts here.

In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
#import folium

print('Libraries imported.')

Libraries imported.


In [11]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Folium imported.')

Folium imported.


In [21]:
#Create a map of Toronto.
map_toronto= folium.Map(location=[43.70011, -79.4163], zoom_start=10)

#for lat, lng,postcode in zip(TorExploredf['Latitude'], TorExploredf['Longitude'], TorExploredf['Postcode']):
#    label= '{}'.format(postcode)
#    label = folium.Popup(label, parse_html=True)
#    folium.CircleMarker(
#         [lat, lng],
#         radius=5,
#         popup=label,
#         color='blue',
#         fill=True,
#         fill_color='#3186cc',
#         fill_opacity=0.7,
#         parse_html=False).add_to(map_toronto)
                        
map_toronto

I could not get labeling to work and time was running out so I decided to move on to some Foursquare exploration.

In [24]:
#Selected the neighborhood of Woburn because it is the same name as a city in Massachusetts. Created a map of Woburn.
map_woburn= folium.Map(location=[43.770992, -79.216917], zoom_start=15)

map_woburn

In [25]:
CLIENT_ID='3TBFBK5V4RYPZUVTQZYUZHYJQNSU5G5NKRK005G5JPQ2CJLM'
CLIENT_SECRET='ONAVWDTDR44E1NSU2211VL2OHUD4G2EA01HTZYBB1VOD1V2L'
VERSION='20180605'
print('Credentials defined')

Credentials defined


In [26]:
#Look for venues in Woburn
LIMIT = 100
radius = 500
url='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, 43.770992, -79.216917, radius, LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?client_id=3TBFBK5V4RYPZUVTQZYUZHYJQNSU5G5NKRK005G5JPQ2CJLM&client_secret=ONAVWDTDR44E1NSU2211VL2OHUD4G2EA01HTZYBB1VOD1V2L&v=20180605&ll=43.770992,-79.216917&radius=500&limit=100'

In [27]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e00d1071d67cb001bf85b43'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.7754920045, 'lng': -79.21069689640106},
   'sw': {'lat': 43.7664919955, 'lng': -79.22313710359893}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4cc1d28c06c254815ac18547',
       'name': 'Starbucks',
       'location': {'address': '300 Borough Dr',
        'crossStreet': 'Scarborough Town Centre',
        'lat': 43.770037201625215,
        'lng': -79.22115586641958,
        'labeledLatLngs':

In [ ]:
#I ran out of time to turn the assignment in so I had to stop analysis here.